<a href="https://colab.research.google.com/github/dpredie/SingleHDR/blob/master/SingleHDR_36.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**[CVPR 2020] Single-Image HDR Reconstruction by Learning to Reverse the Camera Pipeline**

[project website](https://www.cmlab.csie.ntu.edu.tw/~yulunliu/SingleHDR)

In [ ]:
# fetch python code and pre-trained weights
!wget "https://www.cmlab.csie.ntu.edu.tw/~yulunliu/SingleHDR_/code_and_ckpt.zip"
!unzip code_and_ckpt.zip

In [ ]:
%%bash

MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.4-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

In [ ]:
# run single-image HDR reconstruction on the uploaded image
# output: output.hdr
%tensorflow_version 1.x
!pip install --upgrade tensorlayer==1.11.0


In [ ]:
# upload your own LDR image (png or jpg)
import os
from google.colab import files
uploaded = files.upload()
for uploaded_image_name, _ in uploaded.items() :
    print(uploaded_image_name)

Saving R0010004.JPG to R0010004.JPG
R0010004.JPG


In [ ]:

import sys
_ = (sys.path.append("/usr/local/lib/python3.6/site-packages"))

import argparse
import os
import tensorflow as tf
tf.reset_default_graph()
from dequantization_net import Dequantization_net
from linearization_net import Linearization_net
import hallucination_net
from refinement_net import Refinement_net
from util import apply_rf
import numpy as np
import cv2
import glob

_clip = lambda x: tf.clip_by_value(x, 0, 1)

def build_graph(
        ldr,  # [b, h, w, c]
        is_training,
):
    with tf.variable_scope("Dequantization_Net", reuse=tf.AUTO_REUSE):
        dequantization_model = Dequantization_net(is_train=is_training)
        C_pred = _clip(dequantization_model.inference(ldr))

    lin_net = Linearization_net()
    pred_invcrf = lin_net.get_output(C_pred, is_training)
    B_pred = apply_rf(C_pred, pred_invcrf)

    thr = 0.12
    alpha = tf.reduce_max(B_pred, reduction_indices=[3])
    alpha = tf.minimum(1.0, tf.maximum(0.0, alpha - 1.0 + thr) / thr)
    alpha = tf.reshape(alpha, [-1, tf.shape(B_pred)[1], tf.shape(B_pred)[2], 1])
    alpha = tf.tile(alpha, [1, 1, 1, 3])
    with tf.variable_scope("Hallucination_Net", reuse=tf.AUTO_REUSE):
        net_test, vgg16_conv_layers_test = hallucination_net.model(B_pred, 1, False)
        y_predict_test = net_test.outputs
        y_predict_test = tf.nn.relu(y_predict_test)
        A_pred = (B_pred) + alpha * y_predict_test

    # Refinement-Net
    with tf.variable_scope("Refinement_Net", reuse=tf.AUTO_REUSE):
        refinement_model = Refinement_net(is_train=is_training)
        refinement_output = tf.nn.relu(refinement_model.inference(tf.concat([A_pred, B_pred, C_pred], -1)))


    return refinement_output

ldr = tf.placeholder(tf.float32, [None, None, None, 3])
is_training = tf.placeholder(tf.bool)

HDR_out = build_graph(ldr, is_training)


class Tester:

    def __init__(self):
        return

    def test_it(self):
        ldr_img_path = uploaded_image_name
        print(ldr_img_path)
        ldr_img = cv2.imread(ldr_img_path)

        ldr_val = np.flip(ldr_img, -1).astype(np.float32) / 255.0

        ORIGINAL_H = ldr_val.shape[0]
        ORIGINAL_W = ldr_val.shape[1]

        """resize to 64x"""
        if ORIGINAL_H % 64 != 0 or ORIGINAL_W % 64 != 0:
            RESIZED_H = int(np.ceil(float(ORIGINAL_H) / 64.0)) * 64
            RESIZED_W = int(np.ceil(float(ORIGINAL_W) / 64.0)) * 64
            ldr_val = cv2.resize(ldr_val, dsize=(RESIZED_W, RESIZED_H), interpolation=cv2.INTER_CUBIC)
        
        padding = 32
        ldr_val = np.pad(ldr_val, ((padding, padding), (padding, padding), (0, 0)), 'symmetric')

        HDR_out_val = sess.run(HDR_out, {
            ldr: [ldr_val],
            is_training: False,
        })

        HDR_out_val = np.flip(HDR_out_val[0], -1)
        HDR_out_val = HDR_out_val[padding:-padding, padding:-padding]
        if ORIGINAL_H % 64 != 0 or ORIGINAL_W % 64 != 0:
            HDR_out_val = cv2.resize(HDR_out_val, dsize=(ORIGINAL_W, ORIGINAL_H), interpolation=cv2.INTER_CUBIC)
        cv2.imwrite('output.hdr', HDR_out_val)


        return

# ---

sess = tf.Session()
restorer = tf.train.Saver()
restorer.restore(sess, 'ckpt_deq_lin_hal_ref/model.ckpt')

tester = Tester()

tester.test_it()

In [ ]:
# fetch Photomatix for tone mapping
!wget "https://hdrsoft.com/download/linux/PhotomatixCL-6.0.tar.gz"
!tar xf PhotomatixCL-6.0.tar.gz

In [ ]:
# tone map the HDR image
# output: output.jpg
!rm output.jpg
!PhotomatixCL/PhotomatixCL -t1 -o output -d ./ -s jpg output.hdr

In [ ]:
# show the input and output images
import cv2
from google.colab.patches import cv2_imshow
LDR = cv2.imread(uploaded_image_name)
cv2_imshow(LDR)
print('Input LDR image')
print('\n')
HDR = cv2.imread('output.jpg')
cv2_imshow(HDR)
print('Output HDR image')